## 예제2.1 토큰화 코드 

In [2]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


## 예제 2.2 토큰 아이디에서 벡터로 변환

In [3]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)
input_embeddings.shape

torch.Size([1, 5, 16])

In [4]:
input_embeddings

tensor([[[-8.2090e-02,  3.0118e-01, -8.8085e-01, -1.2634e+00,  3.1835e-01,
           1.8315e+00,  1.0709e+00,  4.6994e-01, -6.1449e-01, -7.8928e-01,
          -5.6747e-01,  3.8069e-01,  1.3922e+00,  1.6026e-02,  8.2974e-01,
           1.3334e-02],
         [-1.4649e-01,  2.7216e+00, -1.7160e-01, -2.8612e-01, -1.0756e+00,
           1.8344e-02, -1.5484e+00, -1.9999e-01,  1.4995e+00,  2.8730e-01,
          -2.6202e-01, -2.3328e+00,  4.0326e-01, -1.6367e+00, -1.5895e+00,
           4.6730e-01],
         [ 7.2418e-01,  1.9660e+00,  2.9884e-01,  1.6460e+00, -5.0348e-01,
          -4.6090e-01, -3.0516e-01,  6.7060e-02,  4.8909e-01, -1.9205e-01,
           1.1190e-01, -1.4360e-03,  1.7604e-01,  2.6316e+00, -1.0587e+00,
           7.1069e-01],
         [-2.9339e-01,  5.7470e-01,  1.1450e-01,  1.9807e+00,  8.9175e-01,
          -1.4076e-01,  1.5899e+00,  7.6824e-01,  4.2905e-02,  1.2677e+00,
           1.2534e+00,  6.0061e-01, -3.4329e-01,  1.2633e+00,  6.1302e-01,
          -1.0035e+00],
    

## 예제 2.3 절대적 위치 인코딩

In [5]:
embedding_dim = 16
max_position = 12
# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0) # (1, 5, 16)
# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

In [6]:
input_embeddings[0]

tensor([[ 2.2164, -2.6540, -0.3728,  0.4010,  0.2391,  2.7689,  0.4036,  1.5855,
          0.9992, -0.0907,  2.5438, -0.0434,  0.5913,  1.7388, -2.5533, -0.1122],
        [ 2.1078, -0.5959,  0.4654,  2.0129,  3.1951, -0.3292,  2.6336,  1.9611,
          3.2691,  1.1975,  1.1649, -1.1297,  0.2203,  2.4346,  0.0380,  0.2719],
        [-2.0715, -0.0432, -0.6583,  0.0806,  3.2948,  0.7272, -0.3248,  3.0583,
         -0.5482, -3.4672, -0.2805, -1.0190,  2.8715, -0.5635, -2.1268,  0.5840],
        [ 1.2855, -0.3127,  0.9564,  0.3809,  0.2481,  0.6808,  0.0245,  0.5302,
          0.6166, -1.6801,  0.7439, -2.5479, -1.3264, -1.7036, -0.9039,  0.2451],
        [-0.7249, -0.0696, -0.1815,  0.9022, -1.6912,  1.8822, -0.4360,  0.4355,
         -0.6347,  0.1775,  0.3812, -0.3990,  0.9619,  1.2101,  3.8286, -0.7109]],
       grad_fn=<SelectBackward0>)

## 예제 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [7]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)
# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

In [8]:
weight_q

Linear(in_features=16, out_features=16, bias=True)

In [9]:
input_embeddings.shape

torch.Size([1, 5, 16])

In [10]:
querys.shape

torch.Size([1, 5, 16])

## 예제 2.5. 스케일 점곱 방식의 어텐션

In [11]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	# 쿼리의 마지막 차원 크기를 가져옴 (head_dim)
	dim_k = querys.size(-1) # 16
	# 쿼리와 키의 내적을 계산하고 스케일링 수행
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
	# 소프트맥스를 통해 어텐션 가중치 계산
	weights = F.softmax(scores, dim=-1)
	# 가중치와 값을 곱해 최종 어텐션 결과 반환
	return weights @ values

## 예제 2.6. 어텐션 연산의 입력과 출력

In [12]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)
# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


## 예제 2.7. 어텐션 연산을 수행하는 AttentionHead 클래스

In [13]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

## 예제 2.8. 멀티 헤드 어텐션 구현

In [14]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    # 헤드의 개수와 인과성 마스킹 여부를 저장
    self.n_head = n_head
    self.is_causal = is_causal
    # 쿼리, 키, 값 벡터를 생성하기 위한 선형 층
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    # 멀티 헤드의 출력을 합치고 변환하기 위한 선형 층
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    # 입력 텐서의 크기를 가져옴 (배치 크기, 시퀀스 길이, 임베딩 차원)
    B, T, C = querys.size()
    # 쿼리, 키, 값 벡터를 각각의 헤드로 분할하고 차원을 재배열
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    # 어텐션 연산 수행
    attention = compute_attention(querys, keys, values, self.is_causal)
    # 어텐션 결과를 원래 차원으로 되돌리고 연결
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    # 최종 선형 변환 적용
    output = self.concat_linear(output)
    return output

# 4개의 어텐션 헤드 사용
n_head = 4
# 멀티헤드 어텐션 객체 생성
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
# 입력에 대해 멀티헤드 어텐션 연산 수행
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

## 예제 2.9. 층 정규화 코드

In [15]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

torch.Size([1, 5, 16])

In [16]:
norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[ 3.9116e-08,  7.4506e-09, -1.4901e-08,  0.0000e+00, -3.7253e-09]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

## 예제 2.10. 피드 포워드 층 코드

In [17]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

## 예제 2.11. 인코더 층

In [18]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x

## 예제 2.12. 인코더 구현

In [ ]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
        output = mod(output)
    return output